Model Heat map -> CAM

https://github.com/jacobgil/pytorch-grad-cam

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from cam import * 

In [ ]:
import torch.nn as nn
from torchvision import datasets, transforms, models
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
import torch.utils.data as data

import torch
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
from torchsummary import summary
import os
import cv2

import random
import math
import copy
import time
import gc

In [ ]:
# FRGCv2
num_class = 557  # depth: 557
model_path = 'E:\\LED3D\\newData_training\\Resnet101\\bestModel.pth'  # Resnet101_focal
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

# Resnet101_focal
model = models.resnet101().to(device)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_class).to(device)
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.eval()

heatmap - GradCAM define

In [ ]:
# model define
target_layers = [model.layer4[-1]]
# target_layers = [model.layer2[-1]]
# target_layers = [model.layer3[-1]]
# cam = GradCAM(model=model, target_layers=target_layers, use_cuda=device)
cam = EigenGradCAM(model=model, target_layers=target_layers, use_cuda=device)
targets = None

Image Dataset

In [ ]:
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
import os
import csv

image_id = []
image_label = []
label_map = "E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\label_map.csv"

# 開啟 CSV 檔案 讀取label
with open(label_map, newline='',encoding="utf-8-sig") as csvfile:
    
    rows = csv.reader(csvfile)  
    for row in rows:
        image_id.append(row[0])
        image_label.append(row[1])
    
class IMAGE_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.x = []
        self.y = []
        self.transform = transform
        self.num_classes = 0
        #print(self.root_dir.name)
        
        for i, _dir in enumerate(self.root_dir.glob('*')):
            tmp_str = str(os.path.dirname(_dir))+"\\"
            tmp_str = str(_dir).replace(tmp_str, "") 
            for file in _dir.glob('*'):
                self.x.append(file)
                self.y.append(int(image_id.index(tmp_str)))  # 檔名代表label - > 對應的index label
            self.num_classes += 1
            
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        image = Image.open(self.x[index]).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, self.y[index], str(self.x[index])  # img, label, path

UnNormalize

In [ ]:
def UnNormalize(tensor, mean, std):
    unnormalized_tensor = torch.clone(tensor).detach()
    with torch.no_grad():
        for t in unnormalized_tensor:
            for c, m, s in zip(t, mean, std):
                c.mul_(s).add_(m)
    return unnormalized_tensor

針對heatmap區域做梯度計算
找出紋理較粗糙區域

In [ ]:
def grad_cal(mask_array, photo, threshold=10):  # mask_array = at_map
    mask_array = (mask_array > 127)
    
    # 找出mask區域
    non_zero_indices = np.nonzero(mask_array)  # 找到非零值的索引
    min_row, min_col = np.min(non_zero_indices[0]), np.min(non_zero_indices[1])  # 找到最小行和列
    max_row, max_col = np.max(non_zero_indices[0]), np.max(non_zero_indices[1])  # 找到最大行和列

    # 建立全黑圖
    blank_image = Image.new('L', photo.size, 0)
    photo_array = np.array(photo)

    # mask 對應原圖的區域
    cropped_photo = photo_array[min_row:max_row+1, min_col:max_col+1]

    # Sobel 算子計算圖片的梯度
    sobel_x = cv2.Sobel(cropped_photo, cv2.CV_64F, 1, 0, ksize=3)  # 水平方向的梯度
    sobel_y = cv2.Sobel(cropped_photo, cv2.CV_64F, 0, 1, ksize=3)  # 垂直方向的梯度

    # 計算梯度大小
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    # 根據閾值將梯度幅度影像進行二值化
    binary_gradient = np.where(gradient_magnitude > threshold, 255, 0)
    
    # 把運算好結果貼回全黑區域
    cropped_binary_gradient_image = Image.fromarray(binary_gradient.astype(np.uint8))
    blank_image.paste(cropped_binary_gradient_image, (min_col, min_row))
    
    # 保存結果影像
    # blank_image.save('grad_mask.png')
    
    return blank_image  # grad_mask

生成對抗樣本 (Depth and Normal)

In [ ]:
from fgsm import *
from bim import *
from pgd import *

mean=[0.5, 0.5, 0.5]
std=[0.5, 0.5, 0.5]

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

batch_size = 128

# 0908newData
Depth_dir = 'E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Depth'
Normal_dir = 'E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Normal'

Depth_ds = IMAGE_Dataset(Depth_dir,trans)
Normal_ds = IMAGE_Dataset(Normal_dir,trans)

各個make layer 的 heatmap (測試使用)

In [ ]:
torch.cuda.empty_cache()

cam1 = EigenGradCAM(model=model, target_layers=[model.layer1[-1]], use_cuda=device)
cam2 = EigenGradCAM(model=model, target_layers=[model.layer2[-1]], use_cuda=device)
cam3 = EigenGradCAM(model=model, target_layers=[model.layer3[-1]], use_cuda=device)
targets = None

dataloaders = DataLoader(
    Normal_ds, 
    batch_size= batch_size, 
    pin_memory=False,  
    shuffle=False)

correct = 0
total = 0
start = time.time()
# ssim_value=0

mask_path1 = 'F:\\pytorch-grad-cam-master\\normal_model\\adv_mask_normal50\\mask_layer1'
os.makedirs(mask_path1, exist_ok=True)
mask_path2 = 'F:\\pytorch-grad-cam-master\\normal_model\\adv_mask_normal50\\mask_layer2'
os.makedirs(mask_path2, exist_ok=True)
mask_path3 = 'F:\\pytorch-grad-cam-master\\normal_model\\adv_mask_normal50\\mask_layer3'
os.makedirs(mask_path3, exist_ok=True)


for images, labels, paths in tqdm(dataloaders):
    # at_map = None
    grayscale_cam = cam1(input_tensor=images, targets=targets)
    attention = grayscale_cam * 255
    at_map = torch.from_numpy(attention)  # [batch, width, height]
    
    for i in range(images.shape[0]):
        # save images
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        os.makedirs(os.path.join(mask_path1, str(label)), exist_ok=True)
        img = transforms.ToPILImage()(at_map[i] / 255).convert('L')
        img.save(os.path.join(mask_path1, str(label), name), "PNG")
        
    grayscale_cam = cam2(input_tensor=images, targets=targets)
    attention = grayscale_cam * 255
    at_map = torch.from_numpy(attention)  # [batch, width, height]
    # print('image shape:', images.shape)
    # print('at_map shape:', at_map.shape)
    
    for i in range(images.shape[0]):
        # save images
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        os.makedirs(os.path.join(mask_path2, str(label)), exist_ok=True)
        img = transforms.ToPILImage()(at_map[i] / 255).convert('L')
        img.save(os.path.join(mask_path2, str(label), name), "PNG")
        
    grayscale_cam = cam3(input_tensor=images, targets=targets)
    attention = grayscale_cam * 255
    at_map = torch.from_numpy(attention)  # [batch, width, height]
    # print('image shape:', images.shape)
    # print('at_map shape:', at_map.shape)
    
    for i in range(images.shape[0]):
        # save images
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        os.makedirs(os.path.join(mask_path3, str(label)), exist_ok=True)
        img = transforms.ToPILImage()(at_map[i] / 255).convert('L')
        img.save(os.path.join(mask_path3, str(label), name), "PNG")
            
        
# print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
# print('number of correct: ', float(correct))
#print('ssim_value = {}%'.format(round((ssim_value*100/total), 2)))
end = time.time()
total_time = int(end-start)
m, s = divmod(total_time, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('Spend time: %dd %dh %dm %ds' %(d, h, m, s))

HEATMAP ATTACK

In [ ]:
correct = 0
total = 0
start = time.time()
# ssim_value=0

# Depth_ds \ Normal_ds
dataloaders = DataLoader(
    Depth_ds, 
    batch_size= batch_size, 
    pin_memory=False,  
    shuffle=False)

adv_example_path = 'F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_depth_eps5_alpha0.5'
# mask_path = 'F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_depth\\mask'

os.makedirs(adv_example_path, exist_ok=True)
# os.makedirs(mask_path, exist_ok=True)

if not os.path.isdir(adv_example_path):
    os.mkdir(adv_example_path)
adv_type = ['fgsm', 'bim', 'pgd']

for adv in adv_type:
    if not os.path.isdir(os.path.join(adv_example_path,adv)):
        os.mkdir(os.path.join(adv_example_path,adv))

fgsm_path = os.path.join(adv_example_path, adv_type[0])
bim_path = os.path.join(adv_example_path, adv_type[1])
pgd_path = os.path.join(adv_example_path, adv_type[2])

torch.cuda.empty_cache()

for images, labels, paths in tqdm(dataloaders):
    grayscale_cam = cam(input_tensor=images, targets=targets)
    at_map = grayscale_cam * 255
    
    # 單純 heatmap 用法
    # at_map = torch.from_numpy(at_map)  # [batch, width, height] 轉成tensor
    # for i in range(images.shape[0]):
    #     # save images
    #     label = os.path.basename(paths[i]).split('d')[0]
    #     name = os.path.basename(paths[i])
    #     os.makedirs(os.path.join(mask_path, str(label)), exist_ok=True)
    #     img = transforms.ToPILImage()(at_map[i] / 255).convert('L')
    #     img.save(os.path.join(mask_path, str(label), name), "PNG")
    
    # heatmap + sobel 用法
    for i in range(images.shape[0]):
        # save images
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        # os.makedirs(os.path.join(mask_path, str(label)), exist_ok=True)
        img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Depth', label, name)
        photo = Image.open(img_path)
        at_map[i] = grad_cal(at_map[i], photo, threshold=10)
        # img = Image.fromarray(np.uint8(at_map[i])).convert('L')
        # img.save(os.path.join(mask_path, str(label), name), "PNG")
    at_map = torch.from_numpy(at_map)
    
    # [batch, 3, width, height]
    at_map = (at_map > 127).to(torch.uint8)
    at_map = torch.cat((at_map.unsqueeze(1), at_map.unsqueeze(1), at_map.unsqueeze(1)), dim=1)  # 疊合成三通道 (沒有batch的話疊在dim=0 ; 反之在dim=1)
    
    fgsm_attack = FGSM(model, eps=5/255, at_map=at_map)
    bim_attack = BIM(model, eps=5/255, alpha=0.5/255, steps=10, at_map=at_map)
    pgd_attack = PGD(model, eps=5/255, alpha=0.5/255, steps=40, random_start=True, at_map=at_map)
    
    adv_images_fgsm = fgsm_attack(images, labels)
    adv_images_bim = bim_attack(images, labels)
    adv_images_pgd = pgd_attack(images, labels)
    
    adv_images_fgsm = UnNormalize(adv_images_fgsm, mean=mean, std=std)
    adv_images_bim = UnNormalize(adv_images_bim, mean=mean, std=std)
    adv_images_pgd = UnNormalize(adv_images_pgd, mean=mean, std=std)
    
    for i in range(images.shape[0]):
        img_fgsm = transforms.ToPILImage()(adv_images_fgsm[i]).convert('L')
        img_bim = transforms.ToPILImage()(adv_images_bim[i]).convert('L')
        img_pgd = transforms.ToPILImage()(adv_images_pgd[i]).convert('L')
        
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        
        if not os.path.isdir(fgsm_path + '\\' + str(label)):
           os.mkdir(fgsm_path + '\\' + str(label))
        img_fgsm.save(os.path.join(fgsm_path, str(label), name))
        
        if not os.path.isdir(bim_path + '\\' + str(label)):
           os.mkdir(bim_path + '\\' + str(label))
        img_bim.save(os.path.join(bim_path, str(label), name))
        
        
        if not os.path.isdir(pgd_path + '\\' + str(label)):
           os.mkdir(pgd_path + '\\' + str(label))
        img_pgd.save(os.path.join(pgd_path, str(label), name))
        
    del adv_images_fgsm
    del adv_images_bim
    del adv_images_pgd  
    del at_map 
    gc.collect()
    torch.cuda.empty_cache()
    
        
# print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
# print('number of correct: ', float(correct))
#print('ssim_value = {}%'.format(round((ssim_value*100/total), 2)))
end = time.time()
total_time = int(end-start)
m, s = divmod(total_time, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('Spend time: %dd %dh %dm %ds' %(d, h, m, s))

In [ ]:
torch.cuda.empty_cache()

dataloaders = DataLoader(
    Normal_ds, 
    batch_size= batch_size, 
    pin_memory=False,  
    shuffle=False)

correct = 0
total = 0
start = time.time()
# ssim_value=0

adv_example_path = 'F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_normal_eps5_alpha0.5'
# mask_path = 'F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_normal\\mask'

os.makedirs(adv_example_path, exist_ok=True)
# os.makedirs(mask_path, exist_ok=True)

if not os.path.isdir(adv_example_path):
    os.mkdir(adv_example_path)
adv_type = ['fgsm', 'bim', 'pgd']

for adv in adv_type:
    if not os.path.isdir(os.path.join(adv_example_path,adv)):
        os.mkdir(os.path.join(adv_example_path,adv))

fgsm_path = os.path.join(adv_example_path,'fgsm')
bim_path = os.path.join(adv_example_path,'bim')
pgd_path = os.path.join(adv_example_path,'pgd')

for images, labels, paths in tqdm(dataloaders):
    # at_map = None
    grayscale_cam = cam(input_tensor=images, targets=targets)
    at_map = grayscale_cam * 255
    
    # 單純 heatmap 用法
    # at_map = torch.from_numpy(at_map)  # [batch, width, height] 轉成tensor
    # for i in range(images.shape[0]):
    #     # save images
    #     label = os.path.basename(paths[i]).split('d')[0]
    #     name = os.path.basename(paths[i])
    #     os.makedirs(os.path.join(mask_path, str(label)), exist_ok=True)
    #     img = transforms.ToPILImage()(at_map[i] / 255).convert('L')
    #     img.save(os.path.join(mask_path, str(label), name), "PNG")
    
    # heatmap + sobel 用法
    for i in range(images.shape[0]):
        # save images
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        # os.makedirs(os.path.join(mask_path, str(label)), exist_ok=True)
        img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Normal', label, name)
        photo = Image.open(img_path)
        at_map[i] = grad_cal(at_map[i], photo, threshold=10)
        # img = Image.fromarray(np.uint8(at_map[i])).convert('L')
        # img.save(os.path.join(mask_path, str(label), name), "PNG")
    at_map = torch.from_numpy(at_map)
    
    # [batch, 3, width, height]
    at_map = (at_map > 127).to(torch.uint8)
    at_map = torch.cat((at_map.unsqueeze(1), at_map.unsqueeze(1), at_map.unsqueeze(1)), dim=1)  # 疊合成三通道
    
    fgsm_attack = FGSM(model, eps=5/255, at_map=at_map)
    bim_attack = BIM(model, eps=5/255, alpha=0.5/255, steps=10, at_map=at_map)
    pgd_attack = PGD(model, eps=5/255, alpha=0.5/255, steps=40, random_start=True, at_map=at_map)
    
    adv_images_fgsm = fgsm_attack(model, images, labels, device, eps=4/255, at_map=at_map)
    adv_images_bim = bim_attack(model, images, labels, device, eps=4/255, alpha=0.5/255, iters=10, at_map=at_map)
    adv_images_pgd = pgd_attack(model, images, labels, device, eps=4/255, alpha=0.5/255, at_map=at_map)
    
    adv_images_fgsm = UnNormalize(adv_images_fgsm, mean=mean, std=std)
    adv_images_bim = UnNormalize(adv_images_bim, mean=mean, std=std)
    adv_images_pgd = UnNormalize(adv_images_pgd, mean=mean, std=std)
    
    for i in range(images.shape[0]):
        img_fgsm = transforms.ToPILImage()(adv_images_fgsm[i]).convert('RGB')
        img_bim = transforms.ToPILImage()(adv_images_bim[i]).convert('RGB')
        img_pgd = transforms.ToPILImage()(adv_images_pgd[i]).convert('RGB')
        
        label = os.path.basename(paths[i]).split('d')[0]
        name = os.path.basename(paths[i])
        
        if not os.path.isdir(fgsm_path + '\\' + str(label)):
           os.mkdir(fgsm_path + '\\' + str(label))
        img_fgsm.save(os.path.join(fgsm_path, str(label), name))
        
        if not os.path.isdir(bim_path + '\\' + str(label)):
           os.mkdir(bim_path + '\\' + str(label))
        img_bim.save(os.path.join(bim_path, str(label), name))
        
        
        if not os.path.isdir(pgd_path + '\\' + str(label)):
           os.mkdir(pgd_path + '\\' + str(label))
        img_pgd.save(os.path.join(pgd_path, str(label), name))
        
    del adv_images_fgsm
    del adv_images_bim
    del adv_images_pgd  
    del at_map  
    gc.collect()
    torch.cuda.empty_cache()
        
# print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
# print('number of correct: ', float(correct))
#print('ssim_value = {}%'.format(round((ssim_value*100/total), 2)))
end = time.time()
total_time = int(end-start)
m, s = divmod(total_time, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('Spend time: %dd %dh %dm %ds' %(d, h, m, s))

對抗樣本測試

In [ ]:
from skimage import io
from skimage.metrics import structural_similarity as ssim

# eval

# 改成所需的class
num_class = 557  # depth: 557
model_path = 'E:\\LED3D\\newData_training\\Resnet101\\bestModel.pth'  # Resnet101_focal
batch_size = 64

# Resnet101_focal
model = models.resnet101().to(device)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_class).to(device)
model.load_state_dict(torch.load(model_path, map_location=device), strict=False)
model.eval()
      
mean=[0.5, 0.5, 0.5]
std=[0.5, 0.5, 0.5]

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
depth_path = ['F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_depth_eps5_alpha0.5']
normal_path = ['F:\\pytorch-grad-cam-master\\adv_data\\heatmap_grad_normal_eps5_alpha0.5']
attack = ['fgsm', 'bim', 'pgd']
for idx in depth_path:
    for att in attack:
        Depth_dir = os.path.join(idx, str(att))
        Depth_ds = IMAGE_Dataset(Depth_dir,trans)
        
        Depth_dataloaders = DataLoader(
            Depth_ds, 
            batch_size= batch_size, 
            pin_memory=False,  
            shuffle=False)

        correct = 0
        total = 0
        ssim_score = 0
        wrong = []

        for images, labels, paths in tqdm(Depth_dataloaders):
            
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            _, preds = torch.max(outputs, 1)
            
            total += int(images.shape[0])
            correct += (preds == labels).sum()
            
            for t in range(int(images.shape[0])):
                label = os.path.basename(paths[t]).split('d')[0]
                name = os.path.basename(paths[t])
                img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Depth', label, name)
                image1 = io.imread(paths[t], as_gray=True)  # [256,256,3]
                image2 = io.imread(img_path, as_gray=True)  # [256,256,3]
                ssim_score += ssim(image1, image2, multichannel=False)
                
            #     記錄錯誤 csv
            #     if preds[t] != labels[t]:
            #         wrong.append([paths[t], preds[t], labels[t]])
            # torch.cuda.empty_cache()
            
        print('This is Depth image ' + str(att) + ' attack test:')
        print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
        print('number of correct: ', float(correct))
        print('ssim_value = {}%'.format(round((ssim_score*100/total), 5)))
        print('total: ', int(total))

        # 寫入csv中查看判別錯誤的log
        # csv_name = 'wrong_predict_' + str(att) + '.csv'
        # with open(csv_name, 'w', newline='') as csvfile:
        #     writer = csv.writer(csvfile)

        #     writer.writerow(['path', 'model predict', 'true label'])
        #     # 寫入二維表格
        #     writer.writerows(wrong)
        # del wrong
        
torch.cuda.empty_cache()
print('-------------------------------------')

for idx in normal_path: 
    for att in attack:
        Normal_dir = os.path.join(idx, str(att))
        Normal_ds = IMAGE_Dataset(Normal_dir,trans)

        Normal_dataloaders = DataLoader(
            Normal_ds, 
            batch_size= batch_size, 
            pin_memory=False,  
            shuffle=False)

        correct = 0
        total = 0
        ssim_score = 0
        wrong = []

        for images, labels, paths in tqdm(Normal_dataloaders):
            
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            
            _, preds = torch.max(outputs, 1)
            
            total += int(images.shape[0])
            correct += (preds == labels).sum()
            
            
            for t in range(int(images.shape[0])):
                label = os.path.basename(paths[t]).split('d')[0]
                name = os.path.basename(paths[t])
                img_path = os.path.join('E:\\3D_dataset\\FRGC\\FRGC-2.0-dist\\nd1\\0908newData\\LabelData_Normal', label, name)
                image1 = io.imread(paths[t], as_gray=False)  # [256,256,3]
                image2 = io.imread(img_path, as_gray=False)  # [256,256,3]
                ssim_score += ssim(image1, image2, multichannel=True)
                
                # 記錄錯誤
                # if preds[t] != labels[t]:
                #     wrong.append([paths[t], preds[t], labels[t]])
                
            torch.cuda.empty_cache()
            
        print('This is Normal image ' + str(att) + ' attack test:')    
        print('Accuracy of test text: %f %%' % (100 * float(correct) / total))
        print('number of correct: ', float(correct))
        print('ssim_value = {}%'.format(round((ssim_score*100/total), 5)))
        print('total: ', int(total))

        # 寫入csv中查看判別錯誤的log
        # csv_name = 'wrong_predict_' + str(att) + '.csv'
        # with open(csv_name, 'w', newline='') as csvfile:
        #     writer = csv.writer(csvfile)

        #     writer.writerow(['path', 'model predict', 'true label'])
        #     # 寫入二維表格
        #     writer.writerows(wrong)
        # del wrong
                    